In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from keras.optimizers import Adam

from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_absolute_error

In [2]:
params = {
    'epochs': 200,
    'test_size': 0.2,
    'random_state': 1337,
    'validation_split': 0.2
}

In [3]:
df = pd.read_csv("../data/qsc_out.random_scan_nfp2.csv")

In [4]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,y0,y1,y2,y3,y4,y5,y6
0,-0.104856,0.197790,0.001260,-0.011792,0.000794,0.000120,-0.687180,-0.215177,0.948834,0.264545,0.404154,0.631039,0.898928,1.581192,0.334374
1,-0.110385,0.188144,0.001381,-0.011358,0.000855,0.000138,-0.658897,-0.233486,0.930837,0.298531,0.469811,0.639501,0.876879,1.568947,0.336379
2,-0.095042,0.132051,0.000675,-0.008909,0.000477,0.000348,-0.671871,-0.944641,1.526347,0.385624,0.334515,0.468280,0.804239,1.288313,0.331086
3,-0.115912,-0.207162,0.001411,0.012060,0.000871,-0.000108,-0.736734,0.012462,0.783335,0.278748,0.497138,0.645087,0.926717,1.717088,0.338459
4,-0.096390,-0.217079,0.000933,0.010517,0.000621,-0.000126,-0.764525,-0.048433,0.925351,0.272636,0.548743,0.631384,0.941509,1.549327,0.331432


In [5]:
df.shape

(29674, 15)

In [6]:
x_columns = [col for col in df.columns if col.startswith('x')]
y_columns = [col for col in df.columns if col.startswith('y')]

## ACTUALLY SOLVING THE INVERSE PROBLEM
Y = df[x_columns].values
X = df[y_columns].values

In [7]:
def preprocess_data(X_train, X_test, Y_train, Y_test, params):
    scaler_x = StandardScaler().fit(X_train)
    scaler_y = StandardScaler().fit(Y_train)
    X_train = scaler_x.transform(X_train)
    X_test = scaler_x.transform(X_test)
    Y_train = scaler_y.transform(Y_train)
    Y_test = scaler_y.transform(Y_test)

    input_shape = X_train.shape[1]
    
    output_shape = Y_train.shape[1]
    return X_train, X_test, Y_train, Y_test, input_shape, output_shape, scaler_x, scaler_y

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=params['test_size'], 
                                                    random_state=params['random_state'])

X_train, X_test, Y_train, Y_test, input_shape, output_shape, scaler_x, scaler_y = preprocess_data(X_train, X_test, Y_train, Y_test, params)

In [8]:
X_train.mean(axis=0), X_train.std(axis=0) 

(array([-1.23324421e-14,  2.75938148e-14, -7.59591967e-15, -1.74703992e-14,
        -3.12392412e-14, -5.68256506e-14, -1.97065971e-13]),
 array([1., 1., 1., 1., 1., 1., 1.]))

## Appears to be drift, perhaps the sample is not big enough

In [9]:
X_test.mean(axis=0), X_test.std(axis=0) 

(array([ 0.002863  ,  0.0093879 , -0.00576779, -0.01150026, -0.01160122,
        -0.01466698, -0.0055547 ]),
 array([1.00800321, 1.00616058, 1.0009353 , 0.99290648, 0.99481008,
        1.00920012, 0.71155792]))

In [10]:
Y_train.mean(axis=0), Y_train.std(axis=0) 

(array([-1.86603895e-17, -2.39778990e-17, -4.12750044e-17, -8.97978632e-17,
         1.45237693e-17,  4.88584184e-17, -7.02959790e-15, -1.74744727e-15]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]))

In [11]:
Y_test.mean(axis=0), Y_test.std(axis=0)

(array([ 0.01621399, -0.00112999,  0.01755134,  0.01955565,  0.01673425,
         0.00871709, -0.01095845, -0.02030808]),
 array([0.99488322, 0.99650294, 0.78686316, 0.96749519, 0.93053514,
        0.90833692, 1.01170468, 1.00410648]))

## Dummy regressor

In [12]:
from sklearn.dummy import DummyRegressor

In [13]:
regr = MultiOutputRegressor(DummyRegressor(strategy="mean")).fit(X_train, Y_train)
regr.predict(X_train)

array([[-2.22989316e-17, -3.83122584e-17,  1.19725807e-18, ...,
         4.54958068e-17, -7.02072135e-15, -1.75727554e-15],
       [-2.22989316e-17, -3.83122584e-17,  1.19725807e-18, ...,
         4.54958068e-17, -7.02072135e-15, -1.75727554e-15],
       [-2.22989316e-17, -3.83122584e-17,  1.19725807e-18, ...,
         4.54958068e-17, -7.02072135e-15, -1.75727554e-15],
       ...,
       [-2.22989316e-17, -3.83122584e-17,  1.19725807e-18, ...,
         4.54958068e-17, -7.02072135e-15, -1.75727554e-15],
       [-2.22989316e-17, -3.83122584e-17,  1.19725807e-18, ...,
         4.54958068e-17, -7.02072135e-15, -1.75727554e-15],
       [-2.22989316e-17, -3.83122584e-17,  1.19725807e-18, ...,
         4.54958068e-17, -7.02072135e-15, -1.75727554e-15]])

In [14]:
mean_absolute_error(Y_train, regr.predict(X_train))

0.7520506841719812

In [15]:
mean_absolute_error(Y_test, regr.predict(X_test))

0.7473980441803673

## Train a linear regression for debugging

In [16]:
regr = MultiOutputRegressor(Ridge(random_state=123)).fit(X_train, Y_train)
regr.predict(X_train)

array([[-3.55665513e-02, -2.79101710e-03, -3.97968237e-01, ...,
        -9.39772110e-03,  6.05940341e-01,  4.67702463e-01],
       [ 1.62366516e-02, -5.68281660e-03,  1.66705922e-01, ...,
        -1.90792653e-02, -2.11958735e-01, -8.31790141e-01],
       [ 1.21508354e-01, -8.92874878e-04,  2.18078322e-01, ...,
         8.26661821e-03,  2.45308151e-01,  2.67802098e-01],
       ...,
       [-1.51470886e-01, -7.14298286e-04,  1.08530067e-01, ...,
         1.08681843e-02, -2.90026348e-01,  1.83160605e-01],
       [ 1.23148773e-01, -3.68630926e-03,  4.82192951e-02, ...,
         4.07378792e-03,  1.40822159e-01, -9.40254972e-02],
       [-3.72500527e-02,  3.03308290e-03,  3.33386565e-01, ...,
         8.10580312e-03, -3.00204385e-01,  3.60992209e-01]])

In [17]:
mean_absolute_error(Y_train, regr.predict(X_train))

0.7285670975322824

In [18]:
mean_absolute_error(Y_test, regr.predict(X_test))

0.7245307881968837

## Simplest neural network

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [20]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(input_shape, activation="relu", name="layer_in"),
        layers.Dense(128, activation="relu", name="layer2"),
        layers.Dense(64, activation="relu", name="layer3"),
        layers.Dense(output_shape, name="layer_out"),
    ]
)

model(X_train[0:1])

<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[ 0.48480415,  0.8857344 , -0.61246836, -0.24339984, -0.59986025,
        -0.1226818 , -0.16370966,  0.10784157]], dtype=float32)>

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_in (Dense)            (1, 7)                    56        
                                                                 
 layer2 (Dense)              (1, 128)                  1024      
                                                                 
 layer3 (Dense)              (1, 64)                   8256      
                                                                 
 layer_out (Dense)           (1, 8)                    520       
                                                                 
Total params: 9856 (38.50 KB)
Trainable params: 9856 (38.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanAbsoluteError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanAbsoluteError()],
)

In [23]:
print("Fit model on training data")
history = model.fit(
    X_train,
    Y_train,
    batch_size=64,
    epochs=400,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_test, Y_test),
)

Fit model on training data
Epoch 1/400
371/371 [==============================] - 0s 719us/step - loss: 0.7056 - mean_absolute_error: 0.7056 - val_loss: 0.6842 - val_mean_absolute_error: 0.6842
Epoch 2/400
371/371 [==============================] - 0s 533us/step - loss: 0.6823 - mean_absolute_error: 0.6823 - val_loss: 0.6716 - val_mean_absolute_error: 0.6716
Epoch 3/400
371/371 [==============================] - 0s 523us/step - loss: 0.6723 - mean_absolute_error: 0.6723 - val_loss: 0.6646 - val_mean_absolute_error: 0.6646
Epoch 4/400
371/371 [==============================] - 0s 524us/step - loss: 0.6633 - mean_absolute_error: 0.6633 - val_loss: 0.6564 - val_mean_absolute_error: 0.6564
Epoch 5/400
371/371 [==============================] - 0s 522us/step - loss: 0.6548 - mean_absolute_error: 0.6548 - val_loss: 0.6469 - val_mean_absolute_error: 0.6469
Epoch 6/400
371/371 [==============================] - 0s 520us/step - loss: 0.6477 - mean_absolute_error: 0.6477 - val_loss: 0.6415 - val

Epoch 50/400
371/371 [==============================] - 0s 565us/step - loss: 0.5750 - mean_absolute_error: 0.5750 - val_loss: 0.5828 - val_mean_absolute_error: 0.5828
Epoch 51/400
371/371 [==============================] - 0s 520us/step - loss: 0.5744 - mean_absolute_error: 0.5744 - val_loss: 0.5815 - val_mean_absolute_error: 0.5815
Epoch 52/400
371/371 [==============================] - 0s 645us/step - loss: 0.5738 - mean_absolute_error: 0.5738 - val_loss: 0.5798 - val_mean_absolute_error: 0.5798
Epoch 53/400
371/371 [==============================] - 0s 547us/step - loss: 0.5732 - mean_absolute_error: 0.5732 - val_loss: 0.5810 - val_mean_absolute_error: 0.5810
Epoch 54/400
371/371 [==============================] - 0s 581us/step - loss: 0.5728 - mean_absolute_error: 0.5728 - val_loss: 0.5796 - val_mean_absolute_error: 0.5796
Epoch 55/400
371/371 [==============================] - 0s 536us/step - loss: 0.5722 - mean_absolute_error: 0.5722 - val_loss: 0.5827 - val_mean_absolute_error:

Epoch 99/400
371/371 [==============================] - 0s 561us/step - loss: 0.5564 - mean_absolute_error: 0.5564 - val_loss: 0.5719 - val_mean_absolute_error: 0.5719
Epoch 100/400
371/371 [==============================] - 0s 644us/step - loss: 0.5563 - mean_absolute_error: 0.5563 - val_loss: 0.5729 - val_mean_absolute_error: 0.5729
Epoch 101/400
371/371 [==============================] - 0s 594us/step - loss: 0.5559 - mean_absolute_error: 0.5559 - val_loss: 0.5739 - val_mean_absolute_error: 0.5739
Epoch 102/400
371/371 [==============================] - 0s 559us/step - loss: 0.5558 - mean_absolute_error: 0.5558 - val_loss: 0.5707 - val_mean_absolute_error: 0.5707
Epoch 103/400
371/371 [==============================] - 0s 520us/step - loss: 0.5555 - mean_absolute_error: 0.5555 - val_loss: 0.5698 - val_mean_absolute_error: 0.5698
Epoch 104/400
371/371 [==============================] - 0s 515us/step - loss: 0.5549 - mean_absolute_error: 0.5549 - val_loss: 0.5743 - val_mean_absolute_e

371/371 [==============================] - 0s 519us/step - loss: 0.5463 - mean_absolute_error: 0.5463 - val_loss: 0.5707 - val_mean_absolute_error: 0.5707
Epoch 148/400
371/371 [==============================] - 0s 514us/step - loss: 0.5463 - mean_absolute_error: 0.5463 - val_loss: 0.5691 - val_mean_absolute_error: 0.5691
Epoch 149/400
371/371 [==============================] - 0s 518us/step - loss: 0.5464 - mean_absolute_error: 0.5464 - val_loss: 0.5721 - val_mean_absolute_error: 0.5721
Epoch 150/400
371/371 [==============================] - 0s 516us/step - loss: 0.5460 - mean_absolute_error: 0.5460 - val_loss: 0.5704 - val_mean_absolute_error: 0.5704
Epoch 151/400
371/371 [==============================] - 0s 518us/step - loss: 0.5464 - mean_absolute_error: 0.5464 - val_loss: 0.5692 - val_mean_absolute_error: 0.5692
Epoch 152/400
371/371 [==============================] - 0s 520us/step - loss: 0.5460 - mean_absolute_error: 0.5460 - val_loss: 0.5721 - val_mean_absolute_error: 0.5721


371/371 [==============================] - 0s 522us/step - loss: 0.5401 - mean_absolute_error: 0.5401 - val_loss: 0.5683 - val_mean_absolute_error: 0.5683
Epoch 196/400
371/371 [==============================] - 0s 518us/step - loss: 0.5401 - mean_absolute_error: 0.5401 - val_loss: 0.5694 - val_mean_absolute_error: 0.5694
Epoch 197/400
371/371 [==============================] - 0s 517us/step - loss: 0.5401 - mean_absolute_error: 0.5401 - val_loss: 0.5691 - val_mean_absolute_error: 0.5691
Epoch 198/400
371/371 [==============================] - 0s 517us/step - loss: 0.5398 - mean_absolute_error: 0.5398 - val_loss: 0.5706 - val_mean_absolute_error: 0.5706
Epoch 199/400
371/371 [==============================] - 0s 518us/step - loss: 0.5397 - mean_absolute_error: 0.5397 - val_loss: 0.5687 - val_mean_absolute_error: 0.5687
Epoch 200/400
371/371 [==============================] - 0s 518us/step - loss: 0.5395 - mean_absolute_error: 0.5395 - val_loss: 0.5712 - val_mean_absolute_error: 0.5712


371/371 [==============================] - 0s 537us/step - loss: 0.5352 - mean_absolute_error: 0.5352 - val_loss: 0.5693 - val_mean_absolute_error: 0.5693
Epoch 244/400
371/371 [==============================] - 0s 514us/step - loss: 0.5350 - mean_absolute_error: 0.5350 - val_loss: 0.5701 - val_mean_absolute_error: 0.5701
Epoch 245/400
371/371 [==============================] - 0s 530us/step - loss: 0.5354 - mean_absolute_error: 0.5354 - val_loss: 0.5713 - val_mean_absolute_error: 0.5713
Epoch 246/400
371/371 [==============================] - 0s 570us/step - loss: 0.5346 - mean_absolute_error: 0.5346 - val_loss: 0.5696 - val_mean_absolute_error: 0.5696
Epoch 247/400
371/371 [==============================] - 0s 516us/step - loss: 0.5349 - mean_absolute_error: 0.5349 - val_loss: 0.5751 - val_mean_absolute_error: 0.5751
Epoch 248/400
371/371 [==============================] - 0s 516us/step - loss: 0.5347 - mean_absolute_error: 0.5347 - val_loss: 0.5683 - val_mean_absolute_error: 0.5683


371/371 [==============================] - 0s 650us/step - loss: 0.5316 - mean_absolute_error: 0.5316 - val_loss: 0.5701 - val_mean_absolute_error: 0.5701
Epoch 292/400
371/371 [==============================] - 0s 595us/step - loss: 0.5315 - mean_absolute_error: 0.5315 - val_loss: 0.5685 - val_mean_absolute_error: 0.5685
Epoch 293/400
371/371 [==============================] - 0s 573us/step - loss: 0.5317 - mean_absolute_error: 0.5317 - val_loss: 0.5702 - val_mean_absolute_error: 0.5702
Epoch 294/400
371/371 [==============================] - 0s 523us/step - loss: 0.5315 - mean_absolute_error: 0.5315 - val_loss: 0.5700 - val_mean_absolute_error: 0.5700
Epoch 295/400
371/371 [==============================] - 0s 534us/step - loss: 0.5317 - mean_absolute_error: 0.5317 - val_loss: 0.5689 - val_mean_absolute_error: 0.5689
Epoch 296/400
371/371 [==============================] - 0s 560us/step - loss: 0.5317 - mean_absolute_error: 0.5317 - val_loss: 0.5686 - val_mean_absolute_error: 0.5686


371/371 [==============================] - 0s 522us/step - loss: 0.5289 - mean_absolute_error: 0.5289 - val_loss: 0.5705 - val_mean_absolute_error: 0.5705
Epoch 340/400
371/371 [==============================] - 0s 527us/step - loss: 0.5292 - mean_absolute_error: 0.5292 - val_loss: 0.5679 - val_mean_absolute_error: 0.5679
Epoch 341/400
371/371 [==============================] - 0s 527us/step - loss: 0.5289 - mean_absolute_error: 0.5289 - val_loss: 0.5681 - val_mean_absolute_error: 0.5681
Epoch 342/400
371/371 [==============================] - 0s 522us/step - loss: 0.5289 - mean_absolute_error: 0.5289 - val_loss: 0.5697 - val_mean_absolute_error: 0.5697
Epoch 343/400
371/371 [==============================] - 0s 535us/step - loss: 0.5290 - mean_absolute_error: 0.5290 - val_loss: 0.5698 - val_mean_absolute_error: 0.5698
Epoch 344/400
371/371 [==============================] - 0s 570us/step - loss: 0.5285 - mean_absolute_error: 0.5285 - val_loss: 0.5689 - val_mean_absolute_error: 0.5689


371/371 [==============================] - 0s 520us/step - loss: 0.5266 - mean_absolute_error: 0.5266 - val_loss: 0.5677 - val_mean_absolute_error: 0.5677
Epoch 388/400
371/371 [==============================] - 0s 513us/step - loss: 0.5269 - mean_absolute_error: 0.5269 - val_loss: 0.5690 - val_mean_absolute_error: 0.5690
Epoch 389/400
371/371 [==============================] - 0s 523us/step - loss: 0.5264 - mean_absolute_error: 0.5264 - val_loss: 0.5701 - val_mean_absolute_error: 0.5701
Epoch 390/400
371/371 [==============================] - 0s 518us/step - loss: 0.5266 - mean_absolute_error: 0.5266 - val_loss: 0.5730 - val_mean_absolute_error: 0.5730
Epoch 391/400
371/371 [==============================] - 0s 519us/step - loss: 0.5263 - mean_absolute_error: 0.5263 - val_loss: 0.5705 - val_mean_absolute_error: 0.5705
Epoch 392/400
371/371 [==============================] - 0s 522us/step - loss: 0.5263 - mean_absolute_error: 0.5263 - val_loss: 0.5695 - val_mean_absolute_error: 0.5695


In [24]:
mean_absolute_error(Y_train, model.predict(X_train))

742/742 [==============================] - 0s 305us/step


0.5211591306445805

In [25]:
mean_absolute_error(Y_test, model.predict(X_test))

186/186 [==============================] - 0s 304us/step


0.5678872548686937